In [2]:
import onnx
import numpy as np
import onnx_graphsurgeon as gs

In [3]:
graph1 = gs.import_onnx(onnx.load("weights/webface_batch.onnx"))
output1 = [node for node in graph1.nodes if node.name == "BatchNormalization_126"][0]

In [4]:
graph2 = gs.import_onnx(onnx.load("/workspace/checkpoints/MS1M_Quality_Regression/S4/MFN_net_6epoch.onnx"))

In [5]:
begin = 130
for node in graph2.nodes:
    name = node.name.split("_")[0]
    node.name = name + "_" + str(begin)
    begin+=1

In [6]:
graph2.nodes

[Conv_130 (Conv)
 	Inputs: [
 		Variable (input_1): (shape=['batch', 512, 7, 7], dtype=float32)
 		Constant (37): (shape=[512, 512, 1, 1], dtype=<class 'numpy.float32'>)
 		Constant (38): (shape=[512], dtype=<class 'numpy.float32'>)
 	]
 	Outputs: [
 		Variable (36): (shape=None, dtype=None)
 	]
 Attributes: OrderedDict([('dilations', [1, 1]), ('group', 1), ('kernel_shape', [1, 1]), ('pads', [0, 0, 0, 0]), ('strides', [1, 1])]),
 PRelu_131 (PRelu)
 	Inputs: [
 		Variable (36): (shape=None, dtype=None)
 		Constant (42): (shape=[512, 1, 1], dtype=<class 'numpy.float32'>)
 	]
 	Outputs: [
 		Variable (21): (shape=None, dtype=None)
 	],
 Conv_132 (Conv)
 	Inputs: [
 		Variable (21): (shape=None, dtype=None)
 		Constant (40): (shape=[128, 512, 1, 1], dtype=<class 'numpy.float32'>)
 		Constant (41): (shape=[128], dtype=<class 'numpy.float32'>)
 	]
 	Outputs: [
 		Variable (39): (shape=None, dtype=None)
 	]
 Attributes: OrderedDict([('dilations', [1, 1]), ('group', 1), ('kernel_shape', [1, 1]

In [7]:
output2 = [node for node in graph2.nodes if node.name == "Conv_130"][0]

In [8]:
output2.inputs

[Variable (input_1): (shape=['batch', 512, 7, 7], dtype=float32),
 Constant (37): (shape=[512, 512, 1, 1], dtype=<class 'numpy.float32'>)
 LazyValues (shape=[512, 512, 1, 1], dtype=float32),
 Constant (38): (shape=[512], dtype=<class 'numpy.float32'>)
 LazyValues (shape=[512], dtype=float32)]

In [13]:
identity_out = gs.Variable("identity_out", dtype=np.float32)
identity = gs.Node(op="Identity", inputs=output1.outputs, outputs=[identity_out])
graph1.nodes.append(identity)
graph1.outputs.append(identity_out)

In [14]:
output1.outputs

[Variable (680): (shape=None, dtype=None)]

In [11]:
identity_out = gs.Variable("identity_out", dtype=np.float32)
output1.outputs.append(identity_out)
graph1.outputs.append(identity_out)

In [9]:
output2.inputs[0] = output1.outputs[0]

In [10]:
graph1.nodes.extend(graph2.nodes)
graph1.outputs.append(graph2.outputs[0])

In [11]:
graph1.outputs

[Variable (683): (shape=['batch', 512], dtype=float32),
 Variable (output_1): (shape=['batch', 1], dtype=float32)]

In [12]:
graph1.cleanup().toposort()

[W] colored module is not installed, will not use colors when logging. To enable colors, please install the colored module: python3 -m pip install colored
[E] No function: _ipython_canary_method_should_not_exist_ registered for opset: 11
[W] colored module is not installed, will not use colors when logging. To enable colors, please install the colored module: python3 -m pip install colored
[E] No function: _ipython_display_ registered for opset: 11
[W] colored module is not installed, will not use colors when logging. To enable colors, please install the colored module: python3 -m pip install colored
[E] No function: _ipython_canary_method_should_not_exist_ registered for opset: 11
[W] colored module is not installed, will not use colors when logging. To enable colors, please install the colored module: python3 -m pip install colored
[E] No function: _repr_mimebundle_ registered for opset: 11
[W] colored module is not installed, will not use colors when logging. To enable colors, pleas

Graph torch-jit-export (Opset: 11)
Inputs: [Variable (input.1): (shape=['batch', 3, 112, 112], dtype=float32)]
Nodes:
Conv_0 (Conv)
	Inputs: [
		Variable (input.1): (shape=['batch', 3, 112, 112], dtype=float32)
		Constant (685): (shape=[64, 3, 3, 3], dtype=<class 'numpy.float32'>)
		Constant (686): (shape=[64], dtype=<class 'numpy.float32'>)
	]
	Outputs: [
		Variable (684): (shape=None, dtype=None)
	]
Attributes: OrderedDict([('dilations', [1, 1]), ('group', 1), ('kernel_shape', [3, 3]), ('pads', [1, 1, 1, 1]), ('strides', [1, 1])])
Constant_135 (Constant)
	Inputs: [
	]
	Outputs: [
		Variable (27): (shape=None, dtype=None)
	]
Attributes: OrderedDict([('value', Constant (): (shape=[], dtype=<class 'numpy.int64'>)
LazyValues (shape=[], dtype=int64))])
PRelu_1 (PRelu)
	Inputs: [
		Variable (684): (shape=None, dtype=None)
		Constant (843): (shape=[64, 1, 1], dtype=<class 'numpy.float32'>)
	]
	Outputs: [
		Variable (479): (shape=None, dtype=None)
	]
BatchNormalization_2 (BatchNormalization)

In [ ]:
onnx.checker.check_model(gs.export_onnx(graph1))

In [26]:
onnx.save(gs.export_onnx(graph1), "Web_mod.onnx")